# Imports

In [1]:
import torch
import torch.nn as nn
import bitsandbytes as bnb
from datasets import load_dataset, Dataset
from transformers import (AutoTokenizer,
                          AutoConfig,
                          AutoModelForCausalLM,
                          BitsAndBytesConfig,
                          TrainingArguments)
import transformers
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model
import pandas as pd
from trl import SFTTrainer
from peft.tuners.lora import LoraLayer
import datetime


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/matheusalb/anaconda3/envs/llm/lib/python3.11/site-packages/bitsandbytes/libbitsandbytes_cuda117.so
CUDA SETUP: CUDA runtime path found: /home/matheusalb/anaconda3/envs/llm/lib/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary /home/matheusalb/anaconda3/envs/llm/lib/python3.11/site-packages/bitsandbytes/libbitsandbytes_cuda117.so...


/home/matheusalb/anaconda3/envs/llm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


-----

# Funções

## Carregamento modelo

In [2]:
def create_and_prepare_model(model_name):
    compute_dtype = getattr(torch, "float16")
    
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=compute_dtype,
        bnb_4bit_use_double_quant=True,
    )

    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config=bnb_config,
        device_map={"": 0},
        trust_remote_code=True,
    )
    
    peft_config = LoraConfig(
        lora_alpha=16, 
        lora_dropout=0.1,
        r=64, #64, 32, 16
        bias="none",
        task_type="CAUSAL_LM",
        target_modules=[
            "query_key_value",
            "dense",
            "dense_h_to_4h",
            "dense_4h_to_h",
        ],
    )
    
    tokenizer = AutoTokenizer.from_pretrained(
        model_name,
        trust_remote_code=True
    )
    tokenizer.pad_token = tokenizer.eos_token
    
    return model, peft_config, tokenizer


## Cria dataset

In [3]:
def get_dataset(path, final_string):
    messages = {}
    messages['idSugestaoResposta'] = []
    messages['text'] = [] 
    
    # prompt da versão 1 com falcon instruct, 100 steps
    # gen_text = lambda x: f"### Human: Você é um especialista em responder comentários negativos de um cliente a um restaurante. \
    #             Sua tarefa é responder respeitosamente um comentário negativo de um cliente ao seu restaurante. \
    #             Dado o comentário do cliente entre <>, escreva em Português um comentário de resposta de forma respeitosa, \
    #             empática e não genérica, convencendo o cliente que medidas serão tomadas para resolver o seu problema e \
    #             que ele poderá voltar a fazer pedidos no restaurante. \
    #             Certifique-se de usar detalhes específicos do comentário do cliente.\n \
    #             <{x['comentario']}>\n\
    #             ### Reply: {x['resposta']}"
    gen_text = lambda x: f'''\
Escreva, em Português, um comentário de resposta a ao seguinte comentário de um cliente ao seu restaurante: 
{x['comentario']}
###
{x['sugestaoResposta']}
'''+final_string               
    df = pd.read_csv(path, skip_blank_lines=True)
    df = df.dropna(how='all')
    for _, linha in df.iterrows():
        text = gen_text(linha)
        try:
            int(linha['idSugestaoResposta'])
        except:
            print(linha)
        messages['idSugestaoResposta'].append(int(linha['idSugestaoResposta']))
        messages['text'].append(text)
    
    return Dataset.from_dict(messages)

-----

# Execução

In [5]:
model_name = 'tiiuae/falcon-7b-instruct'
train_path = '../../data/train_base.csv'
validation_path = '../../data/validation_base.csv'    

training_arguments = TrainingArguments(
    output_dir="./results/qlora_falcon7b"+datetime.datetime.now().isoformat(),
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    optim="paged_adamw_32bit",
    save_steps=10,
    logging_steps=10,
    learning_rate=2e-4,
    fp16=True,
    max_grad_norm=0.3,
    max_steps=100, #10000,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
)
model, peft_config, tokenizer = create_and_prepare_model(model_name)
# se não fizer isso vai disparar warning, ativar para inferência 
model.config.use_cache = False

train_data = get_dataset(train_path, tokenizer.eos_token)
validation_data = get_dataset(validation_path, tokenizer.eos_token)

# Supervised finetunning é popularmente conhecido como instruction finetuning
trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=validation_data,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=300,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=False,
)


Loading checkpoint shards: 100%|██████████| 2/2 [00:23<00:00, 11.83s/it]
/home/matheusalb/anaconda3/envs/llm/lib/python3.11/site-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


In [6]:
# import os
# os.environ['WANDB_NOTEBOOK_NAME'] = 'TESTE_2'
trainer.train()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


wandb: Currently logged in as: matheusalb. Use `wandb login --relogin` to force relogin


 10%|█         | 10/100 [00:43<06:14,  4.16s/it]

{'loss': 2.297, 'learning_rate': 0.0002, 'epoch': 0.6}


 20%|██        | 20/100 [01:47<05:51,  4.39s/it]

...

100%|██████████| 100/100 [10:16<00:00,  4.15s/it]

{'loss': 0.6293, 'learning_rate': 0.0002, 'epoch': 5.97}


100%|██████████| 100/100 [10:38<00:00,  6.38s/it]

{'train_runtime': 642.9407, 'train_samples_per_second': 1.244, 'train_steps_per_second': 0.156, 'train_loss': 1.3181130027770995, 'epoch': 5.97}


TrainOutput(global_step=100, training_loss=1.3181130027770995, metrics={'train_runtime': 642.9407, 'train_samples_per_second': 1.244, 'train_steps_per_second': 0.156, 'train_loss': 1.3181130027770995, 'epoch': 5.97})